In [1]:
# END TO END GENERATION OF MARMOT METADATA INFORMATION
# lkp 2023/08/17
#
# Purpose: build a script that will allow a user (Kyle)
# to programmatically generate csv files for upload into
# Scispot LabSheets via bulk import


import pandas as pd
from datetime import date
import numpy as np
import os

In [2]:
## TODO add universal variables here (if there are any)
# future work note: link to GDrive location? S3?

meta_dir = 'G:/My Drive/Lindsay Pino/proj/2023_scispot_utils/data' 

# define Labsheet columns
frx_columns = ["Registry ID",
           "Name",
           "Parent Sample",
           "Cellular Fraction",
           "Post-Treatment Time Point",
           "Time Point Unit",
           "Volume (uL)",
           "Protein concentration (ug/ul)",
           "Harvest Protocol ID",
           "Fractionation Protocol ID",
           "Prepared By",
           "Preparation Date",
           "Record Creator",
           "Storage Location",
           "Plate Barcode",
           "Plate Name",
           "Well Position"]

digest_columns = [
    "Registry ID", 
    "Name", 
    "Parent Sample",
    "Volume (uL)", 
    "Protein Input (ug)", 
    "Spike-In Standard", 
    "Spike-In Amount", 
    "Spike-In Lot ID", 
    "Digest Protocol ID", 
    "Prepared By", 
    "Preparation Date", 
    "Record Creator", 
    "Storage Location", 
    "Plate Barcode", 
    "Plate Name",
    "Well Position"
    ]

msrun_columns = [
    "Registry ID",
    "Run Name",
    "Run Date",
    "Prepared By",
    "Acquisition Type",
    "Gradient Length (Minutes)",
    "Instrument",
    "Instrument LC Type",
    "LC Deck Position",
    "Column ID",
    "Peptide Digest IDs",
    "Injection Volume (uL)",
    "Injection Mass",
    "Injection Mass Unit",
    "Evotip Lot Number",
    "Sample Loading Protocol ID",
    "MS Acquisition Protocol ID",
    "Record Creator",
    "Evotip Rack Barcode",
    "Evotip Rack Name",
    "Tip Position"
]

In [3]:
# Python Script 2: Fractionation
# Register 288 new samples and their plate locations

# hardcoded variables
CELLULAR_FRACTION = ['Nucleoplasm', 'Chromatin', 'Insoluble'] # iterate through each to get the full 3x fractionation
TX_TIME = 4
TX_TIME_UNIT = "Hours"
VOL_FRX = 125
HARVEST_PROTOCOL_ID = "PR003-V1"
FRX_PROTOCOL_ID = "PR004-V1"
PREP_BY = "Bodhi Hueffmeier"
PLATE_BARCODE = "PL0002"
#PLATE_BARCODE = "PL" + str(int(treatment_plate['Plate Barcode'].drop_duplicates()[0][2:]) + 1).rjust(4, '0')
PLATE_NAME = "Set 1 Rep 1"
WELL_POSITION = "" # see below, inherited from previous plate
OUTPUT_SUFFIX = "fractionation"

# ASSUME TREATMENT PLATE HAS ALREADY BEEN GENERATED PER TOY EXAMPLE BELOW
treatment_plate_file = os.path.join(meta_dir, "1K THP-1 Screen - Metadata MVP test case - 01-Cell Culture.csv")
treatment_plate = pd.read_csv(treatment_plate_file)


def cell_fraction_rows(increment, parent_sample, cell_frx):
    name = ""
    well_position = ""
    
    if parent_sample != "NUC1":
        well_position = str(treatment_plate.loc[treatment_plate['Registry ID'] == parent_sample]['Well Position'].iloc[0])
    
    rows = [
    str("FRA" + str(increment)), # Registry ID - don't do anything with   Registry ID,
    name, # Name (temporary field)                   Name,
    parent_sample, # Parent Sample                   Parent Sample, 
    cell_frx, # Cellular Fraction                    Cellular Fraction,
    TX_TIME, # Post-Treatment Time Point             Post-Treatment Time Point,
    TX_TIME_UNIT, # Time Point Unit                  Time Point Unit
    VOL_FRX, # Volume (uL)                               Volume (uL),
    "", # Protein Concentration (ug/uL)              Protein Concentration (ug/uL),
    HARVEST_PROTOCOL_ID, # Harvest Protocol ID       Harvest Protocol ID,
    FRX_PROTOCOL_ID, # Fractionation Protocol ID     Fractionation Protocol ID,
    PREP_BY, # Prepared By                           Prepared By,
    str(date.today()), # Preparation Date            Preparation Date,
    "", # Record Creator                             Record Creator,
    "", # Storage Location                           Storage Location
    PLATE_BARCODE, # Plate Barcode                   Plate Barcode
    str(str(PLATE_NAME) + " " + str(cell_frx)), # Plate Name
    well_position # Well Position
    ]

    return rows

# initialize a new dataframe and begin incrementing placeholder-Registry IDs
frx_df_out = pd.DataFrame(columns=frx_columns)
i = 1
for cell_frx in CELLULAR_FRACTION:
    
    #
    # TODO!!!! 
    # MAP PLATE BARCODES TO INDEX POSITION OF FRACTIONS
    #
    #
    
    new_row = cell_fraction_rows(i, "NUC1", cell_frx) # Generate a row for the 2x cryo nuclei
    new_row[-1] = "A1"
    frx_df_out = frx_df_out.append(pd.DataFrame([new_row], columns=frx_columns)) # add one row for the first cryo nuclei...
    i += 1
    
    new_row[0] =  str("FRA" + str(i))
    new_row[-1] = "A2"
    frx_df_out = frx_df_out.append(pd.DataFrame([new_row], columns=frx_columns)) # ... and a second row for the other cryo nuclei
    i += 1
    
    # now loop through all the "real" samples
    for parent in treatment_plate['Registry ID']: 
        new_row = cell_fraction_rows(i, parent, cell_frx)
        frx_df_out = frx_df_out.append(pd.DataFrame([new_row], columns=frx_columns))
        i += 1
        
    # and finally add the last two wells of NUC1 controls
    new_row = cell_fraction_rows(i, "NUC1", cell_frx) # Generate a row for the 2x cryo nuclei
    new_row[-1] = "H11"
    frx_df_out = frx_df_out.append(pd.DataFrame([new_row], columns=frx_columns)) # add one row for the first cryo nuclei...
    i += 1
    
    new_row[-1] = "H12"
    new_row[0] = str("FRA" + str(i))
    frx_df_out = frx_df_out.append(pd.DataFrame([new_row], columns=frx_columns)) # ... and a second row for the other cryo nuclei
    i += 1

frx_out_filename = str(date.today()) + "-" + PLATE_BARCODE + "-" + OUTPUT_SUFFIX + ".csv"
frx_df_out.to_csv(os.path.join(meta_dir, frx_out_filename), index=False)
del frx_df_out

C:\Users\linds\AppData\Local\Temp\ipykernel_21488\2035712685.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frx_df_out = frx_df_out.append(pd.DataFrame([new_row], columns=frx_columns)) # add one row for the first cryo nuclei...
C:\Users\linds\AppData\Local\Temp\ipykernel_21488\2035712685.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frx_df_out = frx_df_out.append(pd.DataFrame([new_row], columns=frx_columns)) # ... and a second row for the other cryo nuclei
C:\Users\linds\AppData\Local\Temp\ipykernel_21488\2035712685.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  frx_df_out = frx_df_out.append(pd.DataFrame([new_row], columns=frx_columns))
C:\Users\linds\AppData\Local\Temp\ipykernel_21488\2035712685.py:80: 

In [4]:
# Python Script 3: Digestion
# 1) Register digested peptides for chromatin (and nucleoplasm if Rep = 1)

# hardcoded variables

VOL = 170
VOL_POOL = 50
SPIKE_STD = "None"
SPIKE_AMOUNT = ""
SPIKE_LOT = ""
DIGEST_PROTOCOL_ID = "PR005-V1"
PREP_BY = "Julia Robbins"
SET = "1"
REP = "1"
PLATE_BARCODE = "PL0003" 
PLATE_NAME = "Set " + SET + " Rep " + REP + " Digested peptides"
WELL_POSITION = ""
OUTPUT_SUFFIX = "digestion"

# ASSUME FRACTIONATION PLATE HAS ALREADY BEEN GENERATED PER TOY EXAMPLE BELOW
fractionation_plate_file = os.path.join(meta_dir, frx_out_filename)
fractionation_plate = pd.read_csv(fractionation_plate_file)
fractionation_plate

def peptide_digest_rows(increment, parent_sample, well_position, cell_frx):
    
    name = "lol"
    
    name4plate = PLATE_NAME + " (" + cell_frx + ")"
    
    rows = [
    str("PDG" + str(increment)), # Registry ID - don't do anything with   Registry ID,
    name, # Name (temporary field)                   Name,
    parent_sample, # Parent Sample                   Parent Sample,
    VOL, # Volume (uL)                               Volume (uL),
    "", # Protein Input (ug)                         Protein Input (ug),
    SPIKE_STD, # Spike-In Standard                   Spike-In Standard,
    "", # Spike-In Amount (ng)                       Spike-In Amount (ng)
    "", # Spike-In Lot ID                            Spike-In Lot ID 
    DIGEST_PROTOCOL_ID, # Digest Protocol ID         Digest Protocol ID,
    PREP_BY, # Prepared By                           Prepared By,
    str(date.today()), # Preparation Date            Preparation Date,
    "", # Record Creator                             Record Creator,
    "", # Storage Location                           Storage Location
    "", # Plate Barcode
    name4plate, # Plate Name
    well_position # Well position
    ]

    return rows

def peptide_pool_rows(increment, parent_sample, well_position):
    
    name = "lol"
    
    name4plate = str(PLATE_NAME + " (nuclear, dilute)")
    
    rows = [
    str("PDG" + str(increment)), # Registry ID - don't do anything with   Registry ID,
    name, # Name (temporary field)                   Name,
    parent_sample, # Parent Sample                   Parent Sample,
    VOL_POOL, # Volume (uL)                               Volume (uL),
    "", # Protein Input (ug)                         Protein Input (ug),
    "", # Spike-In Standard                   Spike-In Standard,
    "", # Spike-In Amount (ng)                       Spike-In Amount (ng)
    "", # Spike-In Lot ID                            Spike-In Lot ID 
    "", # Digest Protocol ID         Digest Protocol ID,
    PREP_BY, # Prepared By                           Prepared By,
    str(date.today()), # Preparation Date            Preparation Date,
    "", # Record Creator                             Record Creator,
    "", # Storage Location                            Storage Location
    "", # Plate Barcode
    name4plate, # Plate Name
    well_position # Well position
    ]

    return rows

digest_df_out = pd.DataFrame(columns=digest_columns)
i = 1
if REP == "1":
    
    fractionation_plate = fractionation_plate[(fractionation_plate['Cellular Fraction'] == 'Nucleoplasm') |
                                              (fractionation_plate['Cellular Fraction'] == 'Chromatin')]
    
    # now loop through all the "real" samples
    for parent in fractionation_plate['Registry ID']: 
        well_pos = str(fractionation_plate.loc[fractionation_plate['Registry ID'] == parent]['Well Position'].iloc[0])
        cell_fraction = str(fractionation_plate.loc[fractionation_plate['Registry ID'] == parent]['Cellular Fraction'].iloc[0])
        new_row = peptide_digest_rows(i, parent, well_pos, cell_fraction)
        digest_df_out = digest_df_out.append(pd.DataFrame([new_row], columns=digest_columns))
        i += 1
    
    # for Rep 1, we'll also be pooling nuc+chrom for DDA
    # so while we're making these digests, also make the bonus pooled plate
    df_out_pool = pd.DataFrame(columns = digest_columns) 
    
    # hard code NUC1 by plate positions for A1 and A2 plate positions
    temp = fractionation_plate[fractionation_plate['Well Position'] == "A1"]
    well_pos = str(temp['Well Position'].drop_duplicates().iloc[0])
    related_fra = str(', '.join(list(temp['Registry ID'])))
    new_row = peptide_pool_rows(i, related_fra, well_pos)
    df_out_pool = df_out_pool.append(pd.DataFrame([new_row], columns=digest_columns))
    i += 1
    
    temp = fractionation_plate[fractionation_plate['Well Position'] == "A2"]
    well_pos = str(temp['Well Position'].drop_duplicates().iloc[0])
    related_fra = str(', '.join(list(temp['Registry ID'])))
    new_row = peptide_pool_rows(i, related_fra, well_pos)
    df_out_pool = df_out_pool.append(pd.DataFrame([new_row], columns=digest_columns))
    i += 1
    
    for grandparent in fractionation_plate['Parent Sample'].drop_duplicates():
        if grandparent != "NUC1":
            temp = fractionation_plate[fractionation_plate['Parent Sample'] == grandparent]
            well_pos = str(temp['Well Position'].drop_duplicates().iloc[0])
            related_fra = str(', '.join(list(temp['Registry ID'])))
            new_row = peptide_pool_rows(i, related_fra, well_pos)
            df_out_pool = df_out_pool.append(pd.DataFrame([new_row], columns=digest_columns))
            i += 1
        # hard code NUC1 here?
    
    # hard code NUC1 by plate positions for H11 and H12 plate positions
    temp = fractionation_plate[fractionation_plate['Well Position'] == "H11"]
    well_pos = str(temp['Well Position'].drop_duplicates().iloc[0])
    related_fra = str(', '.join(list(temp['Registry ID'])))
    new_row = peptide_pool_rows(i, related_fra, well_pos)
    df_out_pool = df_out_pool.append(pd.DataFrame([new_row], columns=digest_columns))
    i += 1
    
    temp = fractionation_plate[fractionation_plate['Well Position'] == "H12"]
    well_pos = str(temp['Well Position'].drop_duplicates().iloc[0])
    related_fra = str(', '.join(list(temp['Registry ID'])))
    new_row = peptide_pool_rows(i, related_fra, well_pos)
    df_out_pool = df_out_pool.append(pd.DataFrame([new_row], columns=digest_columns))
    i += 1

else:
    
    fractionation_plate = fractionation_plate[(fractionation_plate['Cellular Fraction'] == 'Chromatin')]

    # loop through all the chromatin samples from the fractionation plate to create digests
    for parent in fractionation_plate['Registry ID']: 
        well_pos = str(fractionation_plate.loc[fractionation_plate['Registry ID'] == parent]['Well Position'].iloc[0])
        cell_fraction = "Chromatin"
        new_row = peptide_digest_rows(i, parent, well_pos, cell_fraction)
        digest_df_out = digest_df_out.append(pd.DataFrame([new_row], columns=digest_columns))
        i += 1

#print(df_out.head())
#print(df_out_pool.head())

digest_out_filename = str(date.today()) + "-" + PLATE_BARCODE + "-" + OUTPUT_SUFFIX + ".csv"
out_pool_filename = str(date.today()) + "-" + PLATE_BARCODE + "-" + "pooled-digest" + ".csv"

digest_df_out.to_csv(os.path.join(meta_dir, digest_out_filename), index=False)
df_out_pool.to_csv(os.path.join(meta_dir, out_pool_filename), index=False)
del digest_df_out
del df_out_pool

C:\Users\linds\AppData\Local\Temp\ipykernel_21488\370673969.py:92: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  digest_df_out = digest_df_out.append(pd.DataFrame([new_row], columns=digest_columns))
C:\Users\linds\AppData\Local\Temp\ipykernel_21488\370673969.py:92: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  digest_df_out = digest_df_out.append(pd.DataFrame([new_row], columns=digest_columns))
C:\Users\linds\AppData\Local\Temp\ipykernel_21488\370673969.py:92: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  digest_df_out = digest_df_out.append(pd.DataFrame([new_row], columns=digest_columns))
C:\Users\linds\AppData\Local\Temp\ipykernel_21488\370673969.py:92: FutureWarning: The frame.append method is deprecated and will be remo

In [5]:
# Python Script 5: MS runs
# Register mass spectrometry runs for provided peptides

# hardcoded variables

PREP_BY = "Andrea Gutierrez"
ACQUISITION_TYPE = "DDA" # DDA OR DIA
GRAD_LENGTH = "44"
INSTRUMENT = "timsTOF SCP"
INSTRUMENT_LC = "Evosep"
LC_DECK_POSITION = ""
COLUMN_ID = "MSC1"
INJECT_VOL = "26"
EVOTIP_LOT_NUM = "AB1134"
SAMPLE_LOAD_PROTOCOL = "PR008-V1"
SET = "1"
REP = "1"
PLATE_BARCODE = "PL0004" 
PLATE_NAME = "Set " + SET + " Rep " + REP + ACQUISITION_TYPE + " Evotips"
WELL_POSITION = ""
OUTPUT_SUFFIX = "msruns"

# ASSUME DIGESTION PLATE HAS ALREADY BEEN GENERATED PER TOY EXAMPLE BELOW
digestion_plate_file = os.path.join(meta_dir, digest_out_filename)
digestion_plate = pd.read_csv(digestion_plate_file)
digestion_plate

def ms_run_rows(increment, parent_sample, well_position):
    
    name = "lol"
    
    rows = [
    str("MSR" + str(increment)), # "Registry ID"
    name, # "Run Name",
    str(date.today()), # "Run Date",
    PREP_BY, # "Prepared By",
    ACQUISITION_TYPE, # "Acquisition Type",
    GRAD_LENGTH, # "Gradient Length (Minutes)",
    INSTRUMENT, #"Instrument",
    INSTRUMENT_LC, #"Instrument LC Type",
    LC_DECK_POSITION, # "LC Deck Position",
    COLUMN_ID, # "Column ID"
    parent_sample, # "Peptide Digest IDs",
    INJECT_VOL, # "Injection Volume (uL)",
    "", # "Injection Mass",
    "", # "Injection Mass Unit",
    EVOTIP_LOT_NUM, # "Evotip Lot Number",
    SAMPLE_LOAD_PROTOCOL, # "Sample Loading Protocol ID",
    "", # "MS Acquisition Protocol ID",
    "", # "Record Creator",
    "", # "Evotip Rack Barcode",
    "", # "Evotip Rack Name"
    well_position #"Tip Position"
    ]

    return rows


ms_df_out = pd.DataFrame(columns=msrun_columns)

i = 1
for parent in digestion_plate['Registry ID']: 
    well_pos = str(digestion_plate.loc[digestion_plate['Registry ID'] == parent]['Well Position'].iloc[0])
    new_row = ms_run_rows(i, parent, well_pos)
    ms_df_out = ms_df_out.append(pd.DataFrame([new_row], columns=msrun_columns))
    i += 1

#ms_df_out

msrun_out_filename = str(date.today()) + "-" + PLATE_BARCODE + "-" + OUTPUT_SUFFIX + ".csv"
ms_df_out.to_csv(os.path.join(meta_dir, msrun_out_filename), index=False)
del ms_df_out

C:\Users\linds\AppData\Local\Temp\ipykernel_21488\3334431981.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ms_df_out = ms_df_out.append(pd.DataFrame([new_row], columns=msrun_columns))
C:\Users\linds\AppData\Local\Temp\ipykernel_21488\3334431981.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ms_df_out = ms_df_out.append(pd.DataFrame([new_row], columns=msrun_columns))
C:\Users\linds\AppData\Local\Temp\ipykernel_21488\3334431981.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ms_df_out = ms_df_out.append(pd.DataFrame([new_row], columns=msrun_columns))
C:\Users\linds\AppData\Local\Temp\ipykernel_21488\3334431981.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu